# Guest User and Registered User (People also bought)


In [4]:
import pandas as pd
import numpy as np
import gzip
import json
from lightfm import LightFM
import implicit
from sklearn import metrics
from lightfm.evaluation import auc_score
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k
from scipy.sparse import csr_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/Shared drives/Machine Learning/Recommendation Project/Registered Users/

/content/drive/Shared drives/Machine Learning/Recommendation Project/Registered Users


In [3]:
!pip install lightfm

In [4]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 21.7 MB/s eta 0:00:00


In [8]:
df = pd.read_csv('reviews_2018.csv')

In [8]:
df.head()

,Unnamed: 0.1,Unnamed: 0,overall,reviewerID,asin,Summary,Category,title,price
0,0,0,5.0,A2EM03F99X3RJZ,B00004U9V2,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,"Hand Care: Creams, Lotions , Cleansers & Gloves",Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,$30.00
1,1,1,5.0,A3Z74TDRGD0HU,B00004U9V2,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,"Hand Care: Creams, Lotions , Cleansers & Gloves",Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,$30.00
2,2,2,5.0,A2UXFNW9RTL4VM,B00004U9V2,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,"Hand Care: Creams, Lotions , Cleansers & Gloves",Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,$30.00
3,3,3,5.0,AXX5G4LFF12R6,B00004U9V2,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,"Hand Care: Creams, Lotions , Cleansers & Gloves",Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,$30.00
4,4,4,5.0,A7GUKMOJT2NR6,B00004U9V2,Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,"Hand Care: Creams, Lotions , Cleansers & Gloves",Crabtree &amp; Evelyn - Gardener's Ultra-Moist...,$30.00


In [9]:
top_5_reviewers = df['reviewerID'].value_counts().head(5)
print("Top 5 ReviewerIDs with the most reviews:")
print(top_5_reviewers)


Top 5 ReviewerIDs with the most reviews:
A2GI8X394RLF83    20
A1G60ANYI0HMEU    12
A9BM5GD4ASCMD     12
A2CA0CFUZS89OG    11
AUKZY2RN7AIGM     11
Name: reviewerID, dtype: int64


In [10]:
interaction_matrix = df.pivot_table(index='reviewerID', columns='asin', values='overall', fill_value=0)

In [11]:
interaction_matrix.head()

asin,B00004U9V2,B00005A77F,B00005NDTD,B00005V50B,B00005V50C,B000066SYB,B000068DWY,B00008WFSM,B0000Y3NO6,B0000ZREXG,...,B01HG7OIT8,B01HGFWX4M,B01HGSJNEW,B01HGSJPMW,B01HGSJPWM,B01HHGDG82,B01HIIO7Q4,B01HIQEOLO,B01HIQHQU0,B01HIQIEYC
reviewerID,,,,,,,,,,,,,,,,,,,,,
A01022136U8S5XT86PKU,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0108129TLIKAX34M8AA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0202711ACZFJ9UYUN2T,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0287228OPYR88XXE1AP,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0367965T5IOQH8XLOMR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
sparse_interaction_matrix = csr_matrix(interaction_matrix)

In [16]:
matrix_shape = interaction_matrix.shape

print("Size of Interaction Matrix:", matrix_shape)

Size of Interaction Matrix: (48906, 6176)


In [17]:
train, test = random_train_test_split(sparse_interaction_matrix, test_percentage=0.2, random_state=42)

In [15]:
wmodel = LightFM(learning_rate=0.1, loss='warp')
wmodel.fit(train, epochs=50)

train_precision = precision_at_k(wmodel, train, k=10).mean()
test_precision = precision_at_k(wmodel, test, k=10).mean()

train_auc = auc_score(wmodel, train).mean()
test_auc = auc_score(wmodel, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))


Precision: train 0.11, test 0.01.
AUC: train 1.00, test 0.76.


In [20]:
import joblib

joblib.dump(wmodel, 'wrap_lightfm.pkl')

['wrap_lightfm.pkl']

In [18]:
lmodel = LightFM(learning_rate=0.1, loss='logistic')
lmodel.fit(train, epochs=50)

train_precision = precision_at_k(lmodel, train, k=10).mean()
test_precision = precision_at_k(lmodel, test, k=10).mean()

train_auc = auc_score(lmodel, train).mean()
test_auc = auc_score(lmodel, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.01, test 0.01.
AUC: train 0.86, test 0.81.


In [21]:
import joblib

# Save the re-trained model
joblib.dump(lmodel, 'logistic_lightfm.pkl')


['logistic_lightfm.pkl']

In [22]:
bmodel = LightFM(learning_rate=0.1, loss='bpr')
bmodel.fit(train, epochs=50)

train_precision = precision_at_k(bmodel, train, k=10).mean()
test_precision = precision_at_k(bmodel, test, k=10).mean()

train_auc = auc_score(bmodel, train).mean()
test_auc = auc_score(bmodel, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.08, test 0.00.
AUC: train 0.99, test 0.33.


In [23]:
import joblib
# Save the re-trained model
joblib.dump(bmodel, 'bpr_lightfm.pkl')

['bpr_lightfm.pkl']

In [24]:
wkmodel = LightFM(learning_rate=0.1, loss='warp-kos')
wkmodel.fit(train, epochs=50)

train_precision = precision_at_k(wkmodel, train, k=10).mean()
test_precision = precision_at_k(wkmodel, test, k=10).mean()

train_auc = auc_score(wkmodel, train).mean()
test_auc = auc_score(wkmodel, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.11, test 0.01.
AUC: train 1.00, test 0.76.


In [ ]:
def sample_recommendation_user(model, interactions, user_id, user_dict, item_dict, threshold=0, nrec_items=5, show=True):
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]

    # Predict scores for all items
    scores = pd.Series(model.predict(user_x, np.arange(n_items)))
    scores.index = interactions.columns

    # Sort items by predicted scores in descending order
    scores = list(pd.Series(scores.sort_values(ascending=False).index))

    # Get known items for the user
    known_items = list(pd.Series(interactions.loc[user_id, :][interactions.loc[user_id, :] > threshold].index).sort_values(ascending=False))

    # Remove known items from the recommendation list
    scores = [x for x in scores if x not in known_items]

    # Get the top N recommended items
    return_score_list = scores[0:nrec_items]

    # Map item IDs to their actual names and details
    known_items_details = list(pd.Series(known_items).apply(lambda x: (str(item_dict['asin'][x]), item_dict['title'].get(x, ''), item_dict['price'].get(x, ''))))
    scores_details = list(pd.Series(return_score_list).apply(lambda x: (str(item_dict['asin'][x]), item_dict['title'].get(x, ''), item_dict['price'].get(x, ''))))

    if show:
        print("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i, title, price in known_items_details:
            print(f"{counter}- Title: {title}, Price: {price}")
            counter += 1

        print("\nRecommended Items:")
        counter = 1
        for i, title, price in scores_details:
            print(f"{counter}- Title: {title}, Price: {price}")
            counter += 1


In [ ]:
# Assuming your DataFrame is named df
unique_users = df['reviewerID'].unique()
unique_items = df['asin'].unique()

# Create dictionaries to map user and item IDs to internal IDs
user_dict = {user_id: i for i, user_id in enumerate(unique_users)}
item_dict = {
    'asin': {str(item_id): i for i, item_id in enumerate(unique_items)},
    'title': dict(zip(df['asin'], df['title'])),
    'price': dict(zip(df['asin'], df['price']))
}

# Now you can use these dictionaries in your function
sample_recommendation_user(lmodel, interaction_matrix, user_id='A2GI8X394RLF83', user_dict=user_dict, item_dict=item_dict)


User: A2GI8X394RLF83
Known Likes:
1- Title: Juice Organics Shampoo &amp; Conditioner, Price: $12.00 
2- Title: R+Co Gemstone Color Shampoo, Price: $24.00 
3- Title: Pureology Smooth Perfection Shampoo, 8.5 Fl Oz, Price: $27.50 
4- Title: Replenix Pure Hydration Hyaluronic Acid Serum, Price: $58.80 
5- Title: Macadamia Professional Weightless Moisture Shampoo, Sheer Pecan, Price: $10.00 
6- Title: R+Co Mannequin Styling Paste, Price: $16.00 
7- Title: R+Co Park Ave Blow Out Balm, Price: $29.00 
8- Title: Macadamia Professional Style Extend Dry Shampoo, Price: $19.95 
9- Title: Macadamia Professional Nourishing Moisture Shampoo, Price: $10.00 
10- Title: La Roche-Posay Anthelios AOX Face Sunscreen SPF 50 Daily Antioxidant Face Serum with Sunscreen, 1 Fl. Oz., Price: $42.50 
11- Title: La Roche-Posay Effaclar BB Blur with SPF 20, 1.01 Fl. Oz., Price: $29.99 
12- Title: ORIBE Gold Lust Repair &amp; Restore Shampoo, Price: $49.00 
13- Title: ORIBE Shampoo for Beautiful Color, Price: $16.00 

In [ ]:
import joblib
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from lightfm import LightFM

from lightfm.evaluation import auc_score
from scipy.sparse import csr_matrix



def sample_recommendation_user(model, interactions, user_id, user_dict, item_dict, threshold=0, nrec_items=5, show=True):
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]

    # Predict scores for all items
    scores = pd.Series(model.predict(user_x, np.arange(n_items)))
    scores.index = interactions.columns

    # Sort items by predicted scores in descending order
    scores = list(pd.Series(scores.sort_values(ascending=False).index))

    # Get known items for the user
    known_items = list(pd.Series(interactions.loc[user_id, :][interactions.loc[user_id, :] > threshold].index).sort_values(ascending=False))

    # Remove known items from the recommendation list
    scores = [x for x in scores if x not in known_items]

    # Get the top N recommended items
    return_score_list = scores[0:nrec_items]

    # Map item IDs to their actual names and details
    known_items_details = list(pd.Series(known_items).apply(lambda x: (str(item_dict['asin'][x]), item_dict['title'].get(x, ''), item_dict['price'].get(x, ''))))
    scores_details = list(pd.Series(return_score_list).apply(lambda x: (str(item_dict['asin'][x]), item_dict['title'].get(x, ''), item_dict['price'].get(x, ''))))

    if show:
        print("User: " + str(user_id))
        print("\nRecommended Items:")
        counter = 1
        for i, title, price in scores_details:
            print(f"{counter}- Title: {title}, Price: {price}")
            counter += 1


lightfm_model = joblib.load('logistic_lightfm.pkl')

df = pd.read_csv('reviews_2018.csv')
# Select a user
query = input("Select a User: ")

selected_user = query
interaction_matrix = df.pivot_table(index='reviewerID', columns='asin', values='overall', fill_value=0)
sparse_interaction_matrix = csr_matrix(interaction_matrix)


unique_users = df['reviewerID'].unique()
unique_items = df['asin'].unique()

# Create dictionaries to map user and item IDs to internal IDs
user_dict = {user_id: i for i, user_id in enumerate(unique_users)}
item_dict = {
    'asin': {str(item_id): i for i, item_id in enumerate(unique_items)},
    'title': dict(zip(df['asin'], df['title'])),
    'price': dict(zip(df['asin'], df['price']))
}

# Now you can use these dictionaries in your function
sample_recommendation_user(lightfm_model, interaction_matrix, user_id=selected_user, user_dict=user_dict, item_dict=item_dict)


Select a User: A3KUYXBMJ8AVIX
